# Pixyzの確率分布の記述方法

ここではまず，Pixyzにおける確率モデルの実装方法について説明します．

In [10]:
from __future__ import print_function
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np

torch.manual_seed(1)

In [11]:
from pixyz.utils import print_latex

## 2.1 シンプルな確率分布の設定

ガウス分布を作るためには，`Normal`をインポートして，平均（loc）と標準偏差（scale）を定義します．

In [12]:
from pixyz.distributions import Normal

x_dim = 50
p1 = Normal(loc=torch.tensor(0.), scale=torch.tensor(1.), var=['x'], features_shape=[x_dim], name='p_{1}')

なお``var``には，変数の名前を設定します．ここでは`"x"`を設定しています．

また，dimでは次元数を指定します．ここではdimが50となっていますから，50次元のサンプルを生成する形になります．

上記で定義したp1の情報は次のようにみることができます．

In [13]:
print(p1.distribution_name) 
print(p1.prob_text)

Normal
p_{1}(x)


distribution_nameでは，確率分布の名前を確認できます．

prob_textでは，確率分布の形をテキストで出力できます．ここでテキストに書かれている確率変数は，上記のvarで指定したものです.

また，p1を丸ごとprintすると，以下のように表示されます．

In [14]:
print(p1)

Distribution:
  p_{1}(x)
Network architecture:
  Normal(
    name=p_{1}, distribution_name=Normal,
    var=['x'], cond_var=[], input_var=[], features_shape=torch.Size([50])
    (loc): torch.Size([1, 50])
    (scale): torch.Size([1, 50])
  )


In [15]:
print_latex(p1)

<IPython.core.display.Math object>

次に，定義した分布からサンプリングしてみましょう． サンプリングは，`sample()`によって実行します．

In [16]:
samples = p1.sample()
print(samples)
print(samples["x"])

{'x': tensor([[-1.5256, -0.7502, -0.6540, -1.6095, -0.1002, -0.6092, -0.9798, -1.6091,
         -0.7121,  0.3037, -0.7773, -0.2515, -0.2223,  1.6871,  0.2284,  0.4676,
         -0.6970, -1.1608,  0.6995,  0.1991,  0.8657,  0.2444, -0.6629,  0.8073,
          1.1017, -0.1759, -2.2456, -1.4465,  0.0612, -0.6177, -0.7981, -0.1316,
          1.8793, -0.0721,  0.0663, -0.4370,  0.7626,  0.4415,  1.1651,  2.0154,
          0.2152, -0.5242, -0.1860, -0.6446,  1.5392, -0.8696, -3.3312, -0.7479,
          1.1173,  0.2981]])}
tensor([[-1.5256, -0.7502, -0.6540, -1.6095, -0.1002, -0.6092, -0.9798, -1.6091,
         -0.7121,  0.3037, -0.7773, -0.2515, -0.2223,  1.6871,  0.2284,  0.4676,
         -0.6970, -1.1608,  0.6995,  0.1991,  0.8657,  0.2444, -0.6629,  0.8073,
          1.1017, -0.1759, -2.2456, -1.4465,  0.0612, -0.6177, -0.7981, -0.1316,
          1.8793, -0.0721,  0.0663, -0.4370,  0.7626,  0.4415,  1.1651,  2.0154,
          0.2152, -0.5242, -0.1860, -0.6446,  1.5392, -0.8696, -3.3312, -

出力はdict形式になっています．

サンプリング結果を確認したい変数について指定することで，中身を確認できます（ただし，この例では変数は"x"のみです）．

なお，サンプリング結果は，PyTorchのtensor形式になっています．

続いて，尤度の計算方法を説明します．
例えば，次のサンプルがあったとしましょう．

In [17]:
_sample1 = torch.Tensor([[-0.3030, -1.7618,  0.6348, -0.8044, -1.0371, -1.0669, -0.2085,
         -0.2155,  2.2952,  0.6749,  1.7133, -1.7943, -1.5208,  0.9196,
         -0.5484, -0.3472,  0.4730, -0.4286,  0.5514, -1.5474,  0.7575,
         -0.4068, -0.1277,  0.2804,  1.7460,  1.8550, -0.7064,  2.5571,
          0.7705, -1.0739, -0.2015, -0.5603, -0.6240, -0.9773, -0.1637,
         -0.3582, -0.0594, -2.4919,  0.2423,  0.2883, -0.1095,  0.3126,
         -0.3417,  0.9473,  0.6223, -0.4481, -0.2856,  0.3880, -1.1435,
         -0.6512]])

このサンプルにおける，p1の対数尤度を計算します．

これは，log_likelihoodによって簡単に計算できます．

In [30]:
log_like = p1.log_prob()
print_latex(log_like)

<IPython.core.display.Math object>

In [31]:
log_like.eval({'x': _sample1})

tensor([-72.5003])

なお，log_likelihoodの引数はdictで与えることに注意してください． これは，どの確率変数かを指定するためです（この例では，xしかありませんが）

In [ ]:
# 以下何をしたいのか書く
# 確率の積ができることを示したい

In [32]:
x_dim = 50
_p1 = Normal(loc='mu', scale=torch.tensor(1.), var=['x'], cond_var=['mu'], features_shape=[x_dim])

In [34]:
print(_p1)
print_latex(_p1)

Distribution:
  p(x|\mu)
Network architecture:
  Normal(
    name=p, distribution_name=Normal,
    var=['x'], cond_var=['mu'], input_var=['mu'], features_shape=torch.Size([50])
    (scale): torch.Size([1, 50])
  )


<IPython.core.display.Math object>

In [35]:
_p1.sample({"mu":0})

{'mu': 0,
 'x': tensor([[-0.5962, -1.0055, -0.2106, -0.0075,  1.6734,  0.0103,  0.9837,  0.8793,
          -0.9962, -0.8313, -0.4610, -0.5601,  0.3956, -0.9823,  1.3264,  0.8547,
          -0.6540,  0.7317, -1.4344, -0.5008,  0.1716, -0.1600, -0.5047, -1.4746,
          -1.0412,  0.7323, -1.0483, -0.4709,  0.2911,  1.9907, -0.9247, -0.9301,
           0.8165, -0.9135,  0.2053,  0.3051,  0.5357, -0.4312,  0.1573,  1.2540,
           1.3275, -0.4954, -1.9804,  1.7986,  0.1018,  0.3400, -0.6447, -0.2870,
           3.3212, -0.4021]])}

In [36]:
_p2 = Normal(loc=torch.tensor(0.), scale=torch.tensor(1.), var=['mu'], features_shape=[x_dim])

In [37]:
print(_p2)
print_latex(_p2)

Distribution:
  p(\mu)
Network architecture:
  Normal(
    name=p, distribution_name=Normal,
    var=['mu'], cond_var=[], input_var=[], features_shape=torch.Size([50])
    (loc): torch.Size([1, 50])
    (scale): torch.Size([1, 50])
  )


<IPython.core.display.Math object>

In [ ]:
## 2_3と被ってない？

Pixyzでは分布の積は，掛け算で表すことができます

In [38]:
_p3 = _p1 * _p2
print(_p3) 
print_latex(_p3)

Distribution:
  p(x,\mu) = p(x|\mu)p(\mu)
Network architecture:
  Normal(
    name=p, distribution_name=Normal,
    var=['mu'], cond_var=[], input_var=[], features_shape=torch.Size([50])
    (loc): torch.Size([1, 50])
    (scale): torch.Size([1, 50])
  )
  Normal(
    name=p, distribution_name=Normal,
    var=['x'], cond_var=['mu'], input_var=['mu'], features_shape=torch.Size([50])
    (scale): torch.Size([1, 50])
  )


<IPython.core.display.Math object>

In [ ]:
# sampleのつながりについてより書きたい

In [39]:
_p3.sample()

{'mu': tensor([[-0.3030, -1.7618,  0.6348, -0.8044, -1.0371, -1.0669, -0.2085, -0.2155,
           2.2952,  0.6749,  1.7133, -1.7943, -1.5208,  0.9196, -0.5484, -0.3472,
           0.4730, -0.4286,  0.5514, -1.5474,  0.7575, -0.4068, -0.1277,  0.2804,
           1.7460,  1.8550, -0.7064,  2.5571,  0.7705, -1.0739, -0.2015, -0.5603,
          -0.6240, -0.9773, -0.1637, -0.3582, -0.0594, -2.4919,  0.2423,  0.2883,
          -0.1095,  0.3126, -0.3417,  0.9473,  0.6223, -0.4481, -0.2856,  0.3880,
          -1.1435, -0.6512]]),
 'x': tensor([[-3.2197, -2.3292, -0.5643, -0.8518, -3.0401, -1.5583,  1.5001,  0.5758,
           3.0235,  0.7320,  1.0042, -2.3205, -2.6250,  0.2201,  0.0096, -1.2531,
           2.3094,  0.8959,  0.4809, -1.2004,  0.1038,  1.1519,  0.2723,  2.7226,
           3.2031,  3.6259, -2.7237,  2.9806,  1.3436, -2.8702, -0.5077, -0.9805,
          -0.2331, -0.5901, -0.0193,  0.4189, -2.3976, -3.3211,  1.4084,  1.7670,
           0.1581,  1.0687, -0.9290, -1.1146,  1.0528, -

## 2.2 深層ニューラルネットワークと組み合わせた確率分布の設定

次に， 確率分布のパラメータを深層ニューラルネットワークで定義します．

例えば，ガウス分布の平均$\mu$と分散$\sigma^2$は， パラメータ$\theta$を持つ深層ニューラルネットワークによって，$\mu=f(x;\theta)$および$\sigma^2=g(x;\theta)$と定義できます．

したがって，ガウス分布は${\cal N}(\mu=f(x;\theta),\sigma^2=g(x;\theta))$となります．

Pixyzでは，次のようなクラスを記述することで，これを実現できます．

In [40]:
a_dim = 20

class P2(Normal):
    def __init__(self):
        super(P2, self).__init__(cond_var=['x'], var=['a'])
        
        self.fc1 = nn.Linear(x_dim, 10)
        self.fc21 = nn.Linear(10, a_dim)
        self.fc22 = nn.Linear(10, a_dim)
        
    def forward(self, x):
        h1 = F.relu(self.fc1(x))
        return {'loc': self.fc21(h1), 'scale': F.softplus(self.fc22(h1))}
p2 = P2()

In [ ]:
# loc, scaleとは？？

まず， ガウス分布クラスを継承することで，ガウス分布のパラメータを深層ニューラルネットワークで定義することを明示します．

次に，コンストラクタで，利用するニューラルネットワークを記述します．これは，通常のPyTorchと同じです．

唯一異なる点は，superの引数にvarとcond_varの名前を指定している点です．

varは先程見たように，出力する変数の名前を指定します．一方，cond_varではニューラルネットワークの入力変数の名前を指定します．これは，ここで定義する分布において，条件付けられる変数とみなすことができます．

forwardについても，通常のPyTorchと同じです．ただし，注意点が2つあります．

* 引数の名前と数は，cond_varで設定したものと同じにしてください． 例えば，cond_var=["x", "y"]とした場合は，forward(self, x, y)としてください．ただし，引数の順番は変えても構いません．
* 戻り値は，それぞれの確率分布のパラメータになります．上記の例ではガウス分布なので，平均と分散を指定しています．

そして最後に，定義した確率分布クラスのインスタンスを作成します．

次に，先程の例と同様，確率分布の情報を見てみましょう.

In [41]:
print(p2)
print_latex(p2)

Distribution:
  p(a|x)
Network architecture:
  P2(
    name=p, distribution_name=Normal,
    var=['a'], cond_var=['x'], input_var=['x'], features_shape=torch.Size([])
    (fc1): Linear(in_features=50, out_features=10, bias=True)
    (fc21): Linear(in_features=10, out_features=20, bias=True)
    (fc22): Linear(in_features=10, out_features=20, bias=True)
  )


<IPython.core.display.Math object>

p2の分布は，xで条件付けた形になっています．これらの表記は，superの引数で設定したとおりになっています．

次に，先程の例のように，サンプリングしてみましょう．

注意しなければならないのは，先ほどと異なり，条件づけた変数xがあるということです．

先程おいた_sample1をxとしてサンプリングしましょう．

In [ ]:
## sampleという名前いけてないな，パッとなんのサンプルかわからない

In [42]:
samples = p2.sample({'x': _sample1})
print(samples)
print(samples['a'])
print(samples['x'])

{'x': tensor([[-0.3030, -1.7618,  0.6348, -0.8044, -1.0371, -1.0669, -0.2085, -0.2155,
          2.2952,  0.6749,  1.7133, -1.7943, -1.5208,  0.9196, -0.5484, -0.3472,
          0.4730, -0.4286,  0.5514, -1.5474,  0.7575, -0.4068, -0.1277,  0.2804,
          1.7460,  1.8550, -0.7064,  2.5571,  0.7705, -1.0739, -0.2015, -0.5603,
         -0.6240, -0.9773, -0.1637, -0.3582, -0.0594, -2.4919,  0.2423,  0.2883,
         -0.1095,  0.3126, -0.3417,  0.9473,  0.6223, -0.4481, -0.2856,  0.3880,
         -1.1435, -0.6512]]), 'a': tensor([[-1.0360e+00,  9.8996e-02,  2.5841e-01, -5.8859e-01,  5.1662e-04,
          5.9433e-01, -2.5836e-01,  4.7657e-01, -4.7303e-02,  6.7520e-01,
          2.3160e-01,  2.7852e-01, -1.4851e+00,  3.7574e-01,  1.7171e-01,
          4.6762e-01, -6.4956e-01,  1.0501e-01,  4.8974e-01,  8.8420e-01]])}
tensor([[-1.0360e+00,  9.8996e-02,  2.5841e-01, -5.8859e-01,  5.1662e-04,
          5.9433e-01, -2.5836e-01,  4.7657e-01, -4.7303e-02,  6.7520e-01,
          2.3160e-01,  2.7

出力には，aとxの２つのサンプルがあります．

aが今回計算したサンプルで，xについては，引数として与えたサンプルがそのまま入っています．

次に，尤度計算をします．

尤度計算では， 全ての変数のデータを与える必要があります．

aについては，上記でサンプルした値を入れて計算しましょう．

In [49]:
_sample2 = samples['a']
log_like = p2.log_prob()
print(log_like.eval({'x': _sample1, 'a': _sample2}))
print_latex(log_like)

tensor([-15.8274], grad_fn=<SumBackward2>)


<IPython.core.display.Math object>

なお，これはもっと簡単に書くことができます．

上記のサンプリングで全ての変数とその値がdict形式で出力されたので，それをそのままlog_likelihoodの引数とすればいいのです．

In [50]:
print(log_like.eval(samples))

tensor([-15.8274], grad_fn=<SumBackward2>)


このように記述できる利点は， **変数の数が増えても同じ書き方で尤度計算を実行できる**ことです．

サンプリング->尤度計算という処理は，深層生成モデルでは数多く登場します． このように記述できることで，どのような確率分布の形であっても容易に尤度を計算できるようになるのです．

## 2.3 確率分布の積の設定

最初に書いたとおり，Pixyzの特徴の1つは，確率分布の積を簡単に記述できることです．

ここで，これまで設定した確率分布を再度確認しましょう．

In [53]:
print(p1) 
print_latex(p1)

Distribution:
  p_{1}(x)
Network architecture:
  Normal(
    name=p_{1}, distribution_name=Normal,
    var=['x'], cond_var=[], input_var=[], features_shape=torch.Size([50])
    (loc): torch.Size([1, 50])
    (scale): torch.Size([1, 50])
  )


<IPython.core.display.Math object>

In [54]:
print(p2)
print_latex(p2)

Distribution:
  p(a|x)
Network architecture:
  P2(
    name=p, distribution_name=Normal,
    var=['a'], cond_var=['x'], input_var=['x'], features_shape=torch.Size([])
    (fc1): Linear(in_features=50, out_features=10, bias=True)
    (fc21): Linear(in_features=10, out_features=20, bias=True)
    (fc22): Linear(in_features=10, out_features=20, bias=True)
  )


<IPython.core.display.Math object>

数式的には，これらを掛け合わせることで同時分布を定義できます． 

$p(x,a) = p(a|x)p(x)$

では，Pixyzではこれをどのように記述するのでしょうか．

実は，それぞれの確率分布のインスタンスを**文字通り掛け合わせるだけ**でいいのです．

In [55]:
p3 = p1 * p2

p3ではどのような確率分布が定義されているか確認しましょう．

In [56]:
print(p3)
print_latex(p3)

Distribution:
  p(a,x) = p(a|x)p_{1}(x)
Network architecture:
  Normal(
    name=p_{1}, distribution_name=Normal,
    var=['x'], cond_var=[], input_var=[], features_shape=torch.Size([50])
    (loc): torch.Size([1, 50])
    (scale): torch.Size([1, 50])
  )
  P2(
    name=p, distribution_name=Normal,
    var=['a'], cond_var=['x'], input_var=['x'], features_shape=torch.Size([])
    (fc1): Linear(in_features=50, out_features=10, bias=True)
    (fc21): Linear(in_features=10, out_features=20, bias=True)
    (fc22): Linear(in_features=10, out_features=20, bias=True)
  )


<IPython.core.display.Math object>

確かに，同時分布が定義されていることがわかります．

このインスタンスp3からも，これまでと同様にサンプリングや尤度計算ができます．

In [59]:
samples = p3.sample()
log_like = p3.log_prob().eval(samples)
print(log_like)
print_latex(p3.log_prob())

tensor([-82.8117], grad_fn=<AddBackward0>)


<IPython.core.display.Math object>

In [ ]:
## どういうこと？？

この例をみてわかるように，サンプリングや尤度計算において，もはやp3を構成する各分布の形を気にする必要はありません．

このような記述方法は，Pythonにおける確率モデリングライブラリでは**Pixyzが初めて採用しました**．

これは，確率分布が増えても同じです． 例えば，次の確率分布を定義しましょう．

In [60]:
class P4(Normal):
    def __init__(self):
        super(P4, self).__init__(cond_var=["a", "x"], var=["y"])

        self.fc1 = nn.Linear(x_dim, 10)
        self.fc2 = nn.Linear(a_dim, 10)
        self.fc21 = nn.Linear(10+10, 20)
        self.fc22 = nn.Linear(10+10, 20)

    def forward(self, a, x):
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc2(a))
        h12 = torch.cat([h1, h2], 1)
        return {"loc":self.fc21(h12), "scale":F.softplus(self.fc22(h12))}
    
p4 = P4()

print(p4)
print_latex(p4)

Distribution:
  p(y|a,x)
Network architecture:
  P4(
    name=p, distribution_name=Normal,
    var=['y'], cond_var=['a', 'x'], input_var=['a', 'x'], features_shape=torch.Size([])
    (fc1): Linear(in_features=50, out_features=10, bias=True)
    (fc2): Linear(in_features=20, out_features=10, bias=True)
    (fc21): Linear(in_features=20, out_features=20, bias=True)
    (fc22): Linear(in_features=20, out_features=20, bias=True)
  )


<IPython.core.display.Math object>

p4は，aとxで条件付けられた確率分布です．他に$p(a|x)$，$p(x)$をつかって，同時分布は以下のように書けます．

$p(y,a,x)=p(y|a,x)p(a|x)p(x)$

同時分布が3つの確率分布の積になっていますが，これも上記と同様に書けます．

In [62]:
p5 = p4 * p2 * p1
print(p5)
print_latex(p5)

Distribution:
  p(y,a,x) = p(y|a,x)p(a|x)p_{1}(x)
Network architecture:
  Normal(
    name=p_{1}, distribution_name=Normal,
    var=['x'], cond_var=[], input_var=[], features_shape=torch.Size([50])
    (loc): torch.Size([1, 50])
    (scale): torch.Size([1, 50])
  )
  P2(
    name=p, distribution_name=Normal,
    var=['a'], cond_var=['x'], input_var=['x'], features_shape=torch.Size([])
    (fc1): Linear(in_features=50, out_features=10, bias=True)
    (fc21): Linear(in_features=10, out_features=20, bias=True)
    (fc22): Linear(in_features=10, out_features=20, bias=True)
  )
  P4(
    name=p, distribution_name=Normal,
    var=['y'], cond_var=['a', 'x'], input_var=['a', 'x'], features_shape=torch.Size([])
    (fc1): Linear(in_features=50, out_features=10, bias=True)
    (fc2): Linear(in_features=20, out_features=10, bias=True)
    (fc21): Linear(in_features=20, out_features=20, bias=True)
    (fc22): Linear(in_features=20, out_features=20, bias=True)
  )


<IPython.core.display.Math object>

distributionをみると，p5がどのような分布から構成されているかを表示します． サンプリングや尤度計算も全く同じようにできます．

In [65]:
samples = p5.sample()
log_like = p5.log_prob().eval(samples)
print(log_like)
print_latex(p5.log_prob())

tensor([-109.5795], grad_fn=<AddBackward0>)


<IPython.core.display.Math object>

このように，分布や深層ニューラルネットワークの形を気にせずに，同じ記述方法でサンプリングや尤度計算ができます．
